# Data Collection

## TODO:
- decide regarding the head stuff
- add explanations for analyze_new_data
- maybe add something else?

In [1]:
import os
os.chdir('/app/Pogona_realtime/Arena')

In [2]:
import torch
import glob
import numpy as np
import imp
import matplotlib.pyplot as plt
import cv2 as cv
import pandas as pd
import pickle
import seaborn as sns
import imp
from IPython.display import display, HTML
from sklearn.mixture import GaussianMixture
from scipy import stats
import json
import shutil
import random
import re
import math
from datetime import datetime

from Prediction import dataset
from Prediction import predictor
from Prediction import detector
from Prediction.detector import Detector_v4, Detector_v3
from Prediction import visualize
from Prediction import calibration as calib
from Prediction.kalman_predict import KalmanPredictor
from Prediction.predictor import HitPredictor
from Prediction.visualize import process_video, get_correction_fn, offline_predictor_visualizer
from Prediction import seq2seq_predict
from Prediction import train_eval

WEIGHTS_PATH = 'Prediction/Yolo4/yolo4_gs_best_2306.weights'
CFG_PATH = 'Prediction/Yolo4/yolo4_2306.cfg'

## Initialize YOLOv4 detector

In [3]:
det = Detector_v4()

Detector initiated successfully


## Analyze single trial

In [ ]:
dataset.analyze_rt_data('../../Pogona_Pursuit/Arena/output/feeding_4_20200830-125905', det)

## Analyze new trials

In [ ]:
dataset.analyze_new_data(det)

## Experimental: Convolutional cropped head encoder

### Cropped Head images dataset and dataloader

Detections data pipeline:
- detections.csv -> distortion and homography -> unified dataframe -> pytorch iterable dataset ->
- split each trial to sequences in trial_to_samples (skips sequences with nan - original indexing breaks) ->
- create train and test dataloaders, that returns shuffeled batches of sequences.

Issues with cropped head files:
- Memory issues: in contrast to detections data, not possible to load entire cropped heads data, from hundreds of trials, to memory.
- Distorted and not transformed - we can pass on the distortion, but the homography is important. Maybe only rotate
- Resizing and arranging in PyTorch tensors - possibly the easiest way.
- None entries, aligining with detections indices.
- Currently in a single pickle file, alternative is saving in different 

Options of arranging cropped head data and join on detections data: 
New dataset structure, options. currently, TrajectoriesData dataset has 3D tensors - (# sequence, # index in sequence, # xyxy): 
* __Images in memory__: It's possible to create a new dataset subclass, where it's passed X,Y and another 4d tensor, "X_Images". This tensor could be created by the trial_to_samples function. __Main problem: a single detection's image could be in at most 20 different sequences, thus duplicating the image in memory. Will probably get too heavy, even if the original data is in realtively reasonable size__
* __Paths and alternative \_\_getitem\_\_ implementation__: each single sequence will have an array of paths for the images in that sequence, that will be returned by trial_to_samples. If X is a 3d tensor, then X_paths is a 2D tensor, each row containing either None or the path for the file. In each call to getitem, iterate over the paths and build a 3d tensor, an array of 2d images. Need to save in different files to work. It seems that's the way to do it... "One way of handling this is to split the file into a bunch of smaller files each containing one line of the original, then maintain a list of the paths of these small files and in _getitem_ read the file corresponding to the specified index. _This is how people mostly work with image data, where they have a bunch of images located in a directory_" from https://medium.com/swlh/how-to-use-pytorch-dataloaders-to-work-with-enormously-large-text-files-bbd672e955a0. Also here: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

* Alternatively, we could create new files for each trial, seperate from the dataframe - that natively include both the detections data and the image data, thus making it easier to organize the indices.

* __IterableDataset__: The defualt utils.dataset is a Map Dataset, which supports in memory structures with indexing (or some getitem implementation like the "paths" one). The IterableDataset is supposed to work with data streams (databases, remote servers, very large files, etc.). Doesn't seem to fit our case, becuase we need to 



__Required code changes for small amount of data (without reformatting entire data):__
* Write function to trnasform a list of uneven images to a 3d tensor.
* In trial_to_samples, accept a tensor of images of trial to jointly create a tuple of detections sequences tensor and image sequences tensor
* In TrajectoriesDataset, add images sequences tensor, where getitem returns a sequence of images along with the detections.
* In train_eval function, change to get image tensor and pass it to a network.

In [36]:
imp.reload(train_eval)
imp.reload(dataset)

<module 'Prediction.dataset' from '/app/Pogona_realtime/Arena/Prediction/dataset.py'>

In [37]:
df = dataset.collect_data(data_sources={'detections': True, 'timestamps': True, 'dlc': False, 'touches': True})

46 trials loaded


In [46]:
all_paths = dataset.select_paths(data_sources={'detections': True, 'timestamps': True, 'dlc': False, 'touches': True})

In [39]:
def get_rot_angle(homography):
    return - math.atan2(homography[0,1], homography[0,0]) * 180 / math.pi

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LANCZOS4)
    return result

In [38]:
def get_trial_data_with_heads_that_are_rotated(trial_tuple):
    trial_p = glob.glob(os.path.split(all_paths[trial_tuple]['detections'])[0]+'/*.p')[0]

    with open(all_paths[trial_tuple]['vid_stats'], 'r') as fp:
        vid_stats = json.load(fp)
    
    if vid_stats['homography'] is None:
        with open(dataset.DEFAULT_HOMOGRAPHY_JSON, 'r') as fp:
            h = np.array(json.load(fp)['new_h'])
    else:
        h = np.array(vid_stats['homography']) 
    rot_angle = get_rot_angle(h)

    with open(trial_p, 'rb') as fp:
        head_list = pickle.load(fp)
    
    rotated_head_list = [rotate_image(255-im, rot_angle) if im is not None else None for im in head_list]
    trial_df = df.loc["_".join((trial_tuple[0], str(trial_tuple[1])))]
    
    return trial_df, rotated_head_list

In [40]:
all_trials = df.index.unique()

In [41]:
train, val, test = train_eval.create_train_val_test_splits(all_trials, [0.8, 0.2, 0])

In [67]:
input_labels = ['x1', 'y1', 'x2', 'y2']
output_labels = ['x1', 'y1', 'x2', 'y2']
input_dim = len(input_labels)
output_dim = len(output_labels)

inp_seq_len = 20
out_seq_len = 20
embedding_size = 4  # only used in supporting models.
hidden_size = 300
rnn_layers = 1
dropout = 0.2
batch_size = 128

# train, test
len_list = [4,2]

X_coords_list = [[],[]]

X_images_list = [[],[]]

Y_list = [[],[]]

for i, trials in enumerate([train, val]):
    random.shuffle(trials)
    for trial in trials[:len_list[i]]:
        spl = trial.split('_')
        if spl[-1] == 'None':
            spl[-1] = None
        trial_tup = ('_'.join(spl[:-1]), spl[-1])
        print(trial_tup)
        df_trial, heads = get_trial_data_with_heads_that_are_rotated(trial_tup)

        (X_coords, X_images), Y = train_eval.trial_to_samples(df_trial, input_labels, output_labels, inp_seq_len, out_seq_len, 5, input_images=heads)
        X_coords_list[i].append(X_coords)
        X_images_list[i].append(X_images.float())
        Y_list[i].append(Y)       

        
X_coords_train = torch.cat(X_coords_list[0])
X_coords_test = torch.cat(X_coords_list[1])
X_images_train = torch.cat(X_images_list[0])
X_images_test = torch.cat(X_images_list[1])
Y_train = torch.cat(Y_list[0])
Y_test = torch.cat(Y_list[1])

dataset_train_wh = train_eval.TrajectoriesDataWithHeads(X_coords_train, X_images_train, Y_train)
dataset_test_wh = train_eval.TrajectoriesDataWithHeads(X_coords_test, X_images_test, Y_test)
dataset_train = train_eval.TrajectoriesData(X_coords_train, Y_train)
dataset_test = train_eval.TrajectoriesData(X_coords_test, Y_test)


dl_train_wh = torch.utils.data.DataLoader(dataset_train_wh, batch_size=batch_size, shuffle=True, num_workers=0)
dl_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
dl_test_wh = torch.utils.data.DataLoader(dataset_test_wh, batch_size=batch_size, shuffle=True, num_workers=0)
dl_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=0)

('line_20200803T082002', 'trial1')
('worm_circle_20200830T122241', 'trial1')
('line_20200803T122506', 'trial1')
('circle_20200803T084529', 'trial1')
('cockroach_circle_20200830T123508', 'trial1')
('feeding_4_20200830-125531', None)
